In [1]:
import face_model
import argparse
import cv2
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
import insightface
from insightface.utils import face_align

In [2]:
model = face_model.FaceModel(0,'model-r100-ii/model', 0)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True
loading model-r100-ii/model 0


In [ ]:
img1 = cv2.imread('test images/20.jpg')
img2 = cv2.imread('test images/16.jpg')
image1 = model.get_input(img1)
image2 = model.get_input(img2)

In [ ]:
f1 = model.get_feature(image1)
f2 = model.get_feature(image2)
sim = np.dot(f1, f2)
print(sim)

In [3]:
import insightface
from insightface.utils import face_align

In [4]:
detector = insightface.model_zoo.get_model('retinaface_mnet025_v2')

In [5]:
detector.prepare(ctx_id=0)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [ ]:
def get_input(face_img):
    bbox, pts5 = detector.detect(face_img, threshold=0.8)
    if bbox.shape[0]==0:
        return None
    nimg = face_align.norm_crop(face_img, pts5[0])
    return nimg

In [ ]:
t1= time.time()
ima = cv2.imread('test images/72.jpg')
bbox, pts5 = detector.detect(ima, threshold=0.8)
t2 = time.time()
print(t2-t1)

In [ ]:
bbox[0,0:4]

In [ ]:
for i in bbox:
    print(i[0:4])

In [ ]:
imd = get_input(ima)

In [ ]:
imd.shape

In [ ]:
t1 = time.time()
aligned_faces =[]
for i in range(len(pts5)):
    nimg = face_align.norm_crop(ima, pts5[i])
    aligned_faces.append(nimg)
t2 = time.time()
print(t2-t1)

In [ ]:
len(aligned_faces)

In [ ]:
plt.imshow(aligned_faces[2])

In [ ]:
def detect_face(image_path):
    t0 = time.time()
    image = cv2.imread(image_path)
    bbox, pts5 = detector.detect(image, threshold=0.8)
    mul = []
    fac = dict()
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        la = [int(x1),int(y1),int(x2),int(y2)]
        fac[i] = la
        w,h = (x2-x1),(y2-y1)
        mul.append(w*h)
    ma = mul.index(max(mul))
    lb = fac[ma]
    cv2.rectangle(image,(lb[0], lb[1]),(lb[2],lb[3]), (80,18,236), 2)
    t1 = time.time()
    print(f"took {round(t1-t0, 3)} to get {len(bbox)} faces")
    cv2.imshow('Face Detection',image)
    cv2.waitKey(0)    
    cv2.destroyAllWindows()

In [ ]:
detect_face('test images/38.jpg')

In [ ]:
def detect_face2(image_path):
    t0 = time.time()
    image = cv2.imread(image_path)
    bbox, pts5 = detector.detect(image, threshold=0.8)
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        cv2.rectangle(image,(x1, y1),(x2,y2), (80,18,236), 2)
    t1 = time.time()
    print(f"took {round(t1-t0, 3)} to get {len(bbox)} faces")
    cv2.imshow('Face Detection',image)
    cv2.waitKey(0)    
    cv2.destroyAllWindows()

In [ ]:
detect_face2('test images/38.jpg')

In [ ]:
err_list = []
def find_embedding(image_address):
    img = cv2.imread(image_address)
    bbox, pts5 = detector.detect(img, threshold=0.8)
    if bbox.size ==0:
        err_list.append(image_address)
        pass
    mul = []
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        w,h = (x2-x1),(y2-y1)
        mul.append(w*h)
    ma = mul.index(max(mul))
    nimg = face_align.norm_crop(img, pts5[ma])
    emb = model.get_feature(nimg)
    return emb

In [31]:
import os
from os import listdir

In [ ]:
g = 0
l2 = listdir('M.P. of india')
for i in l2:
    k2 = listdir('M.P. of india/'+i)
    for j in k2:
        fe = find_embedding('M.P. of india/'+i+'/'+j)
        Embedding_and_ID[int(i[-7:])] = fe
    g = g+1
    print(g)

In [ ]:
len(Embedding_and_ID)

In [6]:
import pickle

In [110]:
with open('Embedding_and_ID' + '.pkl', 'wb') as f:
    pickle.dump(Embedding_and_ID, f, pickle.HIGHEST_PROTOCOL)

In [111]:
with open('Name_and_ID' + '.pkl', 'wb') as f:
    pickle.dump(Name_and_ID, f, pickle.HIGHEST_PROTOCOL)

In [112]:
Embedding_and_ID = pickle.load(open('Embedding_and_ID.pkl','rb'))

In [113]:
Name_and_ID = pickle.load(open('Name_and_ID.pkl','rb'))

In [71]:
Name_and_ID[9939983]

'Mukhtar Abbas Naqvi'

In [72]:
len(Name_and_ID)

708

In [73]:
len(Embedding_and_ID)

711

In [100]:
h = 0
Name_and_ID = {}
Embedding_and_ID = {}
for i in range(len(l1)):
    im = cv2.imread('crop faces/politicians crop faces/'+l1[i])
    embd = model.get_feature(im)
    Name_and_ID[int(l1[i][:-4][-7:])] = l1[i][:-12]
    Embedding_and_ID[int(l1[i][:-4][-7:])] = embd
    print(h)
    h = h+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [57]:
Name_and_ID.pop(4158582)

'Mukhtar Abbas Naqvi'

In [64]:
Embedding_and_ID.pop(4158582)

array([ 8.22155327e-02,  8.77090916e-03, -3.83656882e-02,  4.59888624e-03,
       -6.84276149e-02, -1.50155285e-02,  2.86212098e-02, -2.38561295e-02,
       -4.60001193e-02, -9.03891958e-03,  1.53046520e-02, -3.43017541e-02,
        5.76901995e-03,  1.63943344e-03,  9.81184617e-02,  3.47079411e-02,
       -3.09654139e-02, -2.80645303e-02, -3.29155177e-02,  4.76616733e-02,
       -5.51370606e-02,  1.08172204e-02,  1.08100120e-02, -1.05348259e-01,
       -9.40220281e-02, -1.12792966e-03, -2.55989973e-02, -5.50610945e-03,
       -3.81066911e-02, -6.58653257e-03,  1.69797204e-02,  2.98169926e-02,
        4.37442958e-03, -4.41005360e-03, -3.45908590e-02, -1.70875024e-02,
        1.11586731e-02,  3.34728248e-02, -1.96565799e-02, -6.48610964e-02,
       -3.41306180e-02,  5.73131628e-02,  2.03540176e-02, -6.13853149e-02,
        2.20525288e-03, -1.72259286e-02, -1.41384406e-02,  1.16484147e-02,
       -7.08526373e-02,  2.56038737e-03, -8.22301134e-02, -4.96635330e-04,
       -2.20470987e-02, -

In [9]:
import random

In [55]:
def add_face(image_address):
    name = input(str('Enter Name: '))
    ra = random.randint(1000000,9999999)
    img = cv2.imread(image_address)
    bbox, pts5 = detector.detect(img, threshold=0.8)
    if bbox.size ==0:
        print('face not found')
    else:
        mul = []
        for i in range(len(bbox)):
            x1, y1, x2, y2 = bbox[i,0:4]
            w,h = (x2-x1),(y2-y1)
            mul.append(w*h)
        ma = mul.index(max(mul))
        nimg = face_align.norm_crop(img, pts5[ma])
        emb = model.get_feature(nimg)
        Name_and_ID[ra]= name
        Embedding_and_ID[ra] = emb
        cv2.imwrite('crop faces/politicians crop faces/'+name+'_'+str(ra)+'.jpg',nimg)

In [76]:
add_face('new images/5.jpg')

Enter Name: S. Jayshankar


In [11]:
def match_face(image_address):
    t0 = time.time()
    image = cv2.imread(image_address)    
    bbox, pts5 = detector.detect(image, threshold=0.8)
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        nimg = face_align.norm_crop(image, pts5[i])
        embd = model.get_feature(nimg)
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 1)
        max_sim = {}
        for j in Embedding_and_ID:
            representation = Embedding_and_ID[j]
            similarity = np.dot(embd, representation)
            if similarity>0.4:
                max_sim[j] = similarity
        if len(max_sim)!=0:
            max_val = max(max_sim.values())
            for key, value in max_sim.items():
                if max_val == value:
                    ID = key
            person_name = Name_and_ID[ID]
            print(max_sim[ID])
            print(person_name)
            cv2.putText(image, person_name, (x1, y1-2), 0, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(image, 'unknown', (x1, y1-5), 0, 0.6, (255, 0, 0), 2)
    t4 = time.time()
    print(t4-t0,'sec')
    cv2.imshow('Face Recognition', image)
    cv2.waitKey(0)    
    cv2.destroyAllWindows()

In [114]:
match_face('test images/46.jpg')

0.7531427
Mayawati
0.6796547
Sonia_Gandhi
0.74906886
Tejashwi Yadav
0.40888405
Sitaram_Yechury
0.7697055
H._D._Kumaraswamy
0.6470547
Ajit_Singh
0.67368567
Rahul_Gandhi
0.62637234
Akhilesh_Yadav
0.45780805
Mallikarjun_Kharge
0.59628785
P.K. Kunhalikutty
0.74869704246521 sec


In [28]:
def match_face2(frame):
    image = frame
    bbox, pts5 = detector.detect(frame, threshold=0.8)
    for i in range(len(bbox)):
        x1, y1, x2, y2 = bbox[i,0:4]
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        nimg = face_align.norm_crop(image, pts5[i])
        embd = model.get_feature(nimg)
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 1)
        max_sim = {}
        for j in Embedding_and_ID:
            representation = Embedding_and_ID[j]
            similarity = np.dot(embd, representation)
            if similarity>0.4:
                max_sim[j] = similarity
        if len(max_sim)!=0:
            max_val = max(max_sim.values())
            for key, value in max_sim.items():
                if max_val == value:
                    ID = key
            person_name = Name_and_ID[ID]
            print(max_sim[ID])
            print(person_name)
            cv2.putText(image, person_name, (x1, y1-2), 0, 0.5, (0, 0, 255), 2)
        else:
            cv2.putText(image, 'unknown', (x1, y1-5), 0, 0.6, (255, 0, 0), 2)
    if image.shape[1]> 400:
        im2 = cv2.resize(image,(int(image.shape[1]*0.6),int(image.shape[0]*0.6)))
    return im2

In [29]:
def face_recognition_vid(video_address):
    filename = video_address
    dir_name = os.path.splitext(filename)[0]
    parent_dir = "video output/"
    directory = os.path.join(parent_dir, dir_name[12:]) 
    os.mkdir(directory)
    seconds = 1
    cap = cv2.VideoCapture(filename)

    # Get the frames per second
    fps = round(cap.get(cv2.CAP_PROP_FPS)) 
    multiplier = fps*seconds

    # Get the total numer of frames in the video.
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("FC :", frame_count)

    count = -1

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            frame2 = match_face2(frame)
            #cv2.imwrite('F:/Output/%s/Frame{:f}.jpg'.format(count),frame)
            cv2.imwrite('video output/%s/image{:f}.jpg'.format(count)%(dir_name[12:]),frame2)
            count += multiplier 
            cap.set(1, count)
        else:
            cap.release()
            break

In [115]:
face_recognition_vid('test videos/test_video2.mp4')

FC : 1187.0
0.6967752
Arjun Ram Meghwal
0.7001139
Narendra_Modi
0.4165361
Pralhad Joshi
0.6936842
Arjun Ram Meghwal
0.59530044
Narendra_Modi
0.42624795
Pralhad Joshi
0.68954384
Arjun Ram Meghwal
0.58885425
Narendra_Modi
0.60492617
S. Jayshankar
0.5023276
Narendra Singh Tomar
0.41822714
Giriraj_Singh
0.54585105
Prakash javedkar
0.45708436
Ram_Vilas_Paswan
0.5037453
Giriraj_Singh
0.5728849
S. Jayshankar
0.4597425
Narendra Singh Tomar
0.5566251
Prakash javedkar
0.49878097
Ram_Vilas_Paswan
0.49806583
Giriraj_Singh
0.6156489
S. Jayshankar
0.5431898
Narendra Singh Tomar
0.5286195
Prakash javedkar
0.5095228
Ram_Vilas_Paswan
0.5368206
Giriraj_Singh
0.5877502
S. Jayshankar
0.4911734
Narendra Singh Tomar
0.4790306
Ram_Vilas_Paswan
0.53406644
Prakash javedkar
0.5471773
Giriraj_Singh
0.58763254
Narendra Singh Tomar
0.6471973
S. Jayshankar
0.58379936
Prakash javedkar
0.44926357
Ram_Vilas_Paswan
0.4455506
Giriraj_Singh
0.61956435
S. Jayshankar
0.47346333
Narendra Singh Tomar
0.4852299
Ram_Vilas_Pasw